# Исследование надёжности заёмщиков

### Описание исследования

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга**.

### Цель исследования.
    
Суть исследования: 

Провести анализ данных при помощи которых банк разработает модель **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
    
_____
### Задачи исследования.

1. Выявить, есть ли зависимость между показателями:
* количество детей и возвратом кредита в срок;
* семейным положением и возвратом кредита в срок;
* уровнем дохода и возвратом кредита в срок;
2. Проанализировать, как ак разные цели кредита влияют на его возврат в срок?
_____
### Исходные данные.
    
Данные о клиентах банка.
______

#### Данное исследование разделим на несколько частей.

### Часть 1. Изучение общей информации:
* [1. Изучение файлов с данными, получение общей информации, загрузка библиотек.](#1-bullet)
* [2. Нахождение и ликвидация пропусков.](#2-bullet)

### Часть 2. Подготовка данных:
* [1. Приведение данных к нужным типам.](#3-bullet)
* [2. Нахождение и исправление аномалий и ошибок.](#4-bullet)
* [Ответы на вопросы](#5-bullet)

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd


df = pd.read_csv('data.csv')
df.info()
display(df.head())
gaps_data = pd.DataFrame({'null':df.isnull().sum(), '%':df.isnull().sum() / len(df) * 100}) #пропуски в данных и их % от всех данных

gaps_data.sort_values(by='null', ascending=False).round()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,null,%
days_employed,2174,10.0
total_income,2174,10.0
children,0,0.0
dob_years,0,0.0
education,0,0.0
education_id,0,0.0
family_status,0,0.0
family_status_id,0,0.0
gender,0,0.0
income_type,0,0.0


**Вывод**

В таблице 12 столбцов, согласно заданию:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

1. В столбцах "days_employed" и "total_income" пропущены значения.
Пропуски неслучаные, так как есть взаимосвязь между пропусками в колонках, вероятность пропуска зависит от других значений, в том числе и от значений собственного столбца. Возможные причины пропусков данных могут быть ошибки ввода данных, сокрытие информации.
2. Тип данных. В столбцах "days_employed" и "total_income" (стаже и доходе) стоят значения типа float (вещественный) их требуется заменить на int (целочисленный).
3. В столбце "education" требуется удаление дубликатов.
4. В "days_employed" (стаже) есть отрицательные значения - их можно заменить на положительные, не удаляя столбец. Следует отметить, что значения стажа для некоторых групп существенно высокие, однако колонка прямо не участвует в расчетах, поэтому носит справочный характер и не требует пересчета. Возможной причиной изменений может быть ошибка предоставления данных, требуется уточнение от сотрудников, предоставивших данные (возможно, в часах вместо дней или отработанных человеко-часах, выгруженных из табеля учета рабочего времени).


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
medians_total_income = df.groupby('income_type')['total_income'].median() #медианные уровни дохода для каждого типа занятости
#замена в группах с нулевыми на медианы по группам занятости
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]

df.loc[df['total_income'] == 0, 'total_income'].value_counts() #проверка замены

df.loc[df['dob_years'] == 0, 'income_type'].value_counts() #ошибочный возраст по типам занятости

age_medians = df.groupby('income_type')['dob_years'].median() #медианный возраст для каждой группы

df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'сотрудник'), 'dob_years'] = age_medians[6]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = age_medians[4]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'компаньон'), 'dob_years'] = age_medians[3]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'госслужащий'), 'dob_years'] = age_medians[2]

df.loc[df['dob_years'] == 0, 'dob_years'].value_counts() #проверка замены


age_statistics = df['dob_years'].describe() #данные статистики, границы распределения совокупности 25%-50%-75%
age_statistics[4:7] #результаты 34 - 43 - 53

def group(age):
    if age <= age_statistics[4]: return 'меньше 34'
    elif age_statistics[4] < age <= age_statistics[5]: 
        return 'от 34 до 43'
    elif age_statistics[5] < age <= age_statistics[6]: 
        return 'от 43 до 53'
    else: return 'старше 53'
    
df['group_age'] = df['dob_years'].apply(group)

medians_days_employed = df.groupby('group_age')['days_employed'].median() #медианные уровни стажа

#замена в группах с нулевыми на медианы по стажу на базе групп в годах

df.loc[(df['days_employed'].isnull()) & (df['group_age'] == 'от 34 до 43'), 'days_employed'] = medians_days_employed[0]
df.loc[(df['days_employed'].isnull()) & (df['group_age'] == 'от 43 до 53'), 'days_employed'] = medians_days_employed[1]
df.loc[(df['days_employed'].isnull()) & (df['group_age'] == 'старше 53'), 'days_employed'] = medians_days_employed[2]
df.loc[(df['days_employed'].isnull()) & (df['group_age'] == 'меньше 34'), 'days_employed'] = medians_days_employed[3]

gaps_data = pd.DataFrame({'null':df.isnull().sum(), '%':df.isnull().sum() / len(df) * 100}) #пропуски в данных и их % от всех данных

gaps_data.sort_values(by='null', ascending=False).round()

,null,%
children,0,0.0
days_employed,0,0.0
dob_years,0,0.0
education,0,0.0
education_id,0,0.0
family_status,0,0.0
family_status_id,0,0.0
gender,0,0.0
income_type,0,0.0
debt,0,0.0


**Вывод**
* Пропуски в стаже и ежемесячном доходе заменены на медианное значение.  Принцип заполнения пропусков - по характерному медианному значению. 
* После обработки данные о стаже стали положительными.

### Замена типа данных

In [5]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
 12  group_age         21525 non-null  object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.0+ MB


**Вывод**
<br> В столбцах "days_employed" и "total_income" (стаже и доходе) значения типа float (вещественный) были заменены на int (целочисленный).

# Обработка дубликатов

In [ ]:
#обработка видимых дубликатов с разным регистром в столбце education
df['education'] = df['education'].str.lower()
#проверим прочие столбцы
print(df['children'].value_counts())
print(df['family_status'].value_counts())
print(df['purpose'].value_counts())
print(df['income_type'].value_counts())
#в данных о детях есть отрицательные (-1) и слишков высокие значения (20, возможно ошибка)
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2) #предположим, что ошибочен только ноль


df.duplicated().sum() #число дубликатов

df = df.drop_duplicates().reset_index(drop = True) #удаление идентичных дубликатов

df.duplicated().sum()

**Вывод**

После обработки дубликатов видно, что данные о целях кредита требуют лемматизации с целью замены для последующей группировки.  
Для категоризации были выделены вручную ключевые слова:
* недвижимость (включающая слова "недвижимость" и "жилье", сюда же была отнесена цель ремонт жилья, тк это неотделимое улучшение залога кредита и часто сопутствует кредитованию на покупку или строительство); 
* образование; 
* автомобиль; 
* свадьба.  
Итоговая категоризация будет содержать две залоговых цели кредита (недвижимость и жилье) и два потребительских (образование и свадьба).

### Лемматизация

In [ ]:
import pandas as pd
from pymystem3 import Mystem # pymystem3 импортируется
m = Mystem()

def lemma_pur(row): #функция лемматизации
    lemma = m.lemmatize(row['purpose'])
    if 'автомобиль' in lemma:
        return "автомобиль" #формируем группы для категорий
    if "образование" in lemma:
        return "образование"
    if "свадьба" in lemma:
        return "свадьба"
    if "недвижимость" or "жилье" in lemma:
        return "недвижимость"

df['purpose'] = df.apply(lemma_pur, axis=1) #не сохраняю исходный столбец со значениями до лемматизации, тк эти данные не нужны в дальнейшем

display(df.head()) 


In [ ]:
df['purpose'].value_counts()

**Вывод**

После лемматизации не только устранены дубликаты, но и сформированы группы для проверки гипотезы о наличии взаимосвязи между целью кредита и вероятностью невыплаты.

### Категоризация данных

In [ ]:
#категоризация по семейному статусу не требуется, 5 категорий сформированы изначально
print(df['family_status'].value_counts())
#категоризация по числу детей не требуется, только по наличию
def got_children(children):
    if children > 0: 
        return "есть дети"
    else: 
        return "нет детей"
    
df['got_children'] = df['children'].apply(got_children)

#категоризация по доходам
statistics = df['total_income'].describe() #данные статистики, границы распределения совокупности 25%-50%-75%
statistics[4:7] #выбираем столбцы с границами для функции, чтобы разделить на 4 группы
def total_income(income):
    if income <= statistics[4]:
        return "низкий"
    elif statistics[4] < income <= statistics[5]:
        return "средний"
    elif statistics[5] < income <= statistics[6]: 
        return "высокий"
    else: 
        return "очень высокий"

df['income_group'] = df['total_income'].apply(total_income)
df.head()

**Вывод**

В результате сформированы группы по уровню доходов (для группировки использовались данные о квантилях нормального распределения 25%-50%-75%):
* `низкий` — клиенты, чей доход менее 103053.15 
* `средний` — клиенты с доходом от 103053.15 до 145017.93
* `высокий` — клиенты с доходом от от 145017.93 до 203435.06
* `очень высокий` — клиенты с доходом выше 203435.06
Сформированы категории клиентов по наличию детей (есть дети / нет детей).

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#построим сводную таблицу, групп всего две, поэтому столбец для расчета отношения не понадобится
data_pivot = round(df.pivot_table(index = ['got_children'], values = 'debt') * 100, 1)
data_pivot

**Вывод**  
Зависимость между наличием детей и возвратом кредита в срок есть: при отсутствии детей ниже вероятность кредитной задолженности (7,5%), чем при их наличии (9,2%)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#построим сводную таблицу
data_pivot = df.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count').round(2)
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]) * 100, 1)
data_pivot.sort_values('ratio', ascending = False)

**Вывод**

Зависимость между семейным положением и возвратом кредита в срок есть: 
* вероятность невыплаты ниже (и составляет 7,5%), если клиент находится в браке, чем если он не женат (не замужем) (9,8%) или состоит в незарегистрированном браке (9,3%).  
В тоже время после семейных отношений ответственность не снижается, а растет:
* для клиентов, вдовцов/вдовиц - 6,6%;
* для клиентов, находящихся в разводе - 7,1%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#построим сводную таблицу
data_pivot = df.pivot_table(index = ['income_group'], columns = 'debt', values = 'gender', aggfunc = 'count').round(2)
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]) * 100, 1)
data_pivot.sort_values('ratio', ascending = False)

**Вывод**

Гипотеза о зависимости между уровнем дохода и возвратом кредита в срок опровергнута.  
 Клиенты с высоким и средним доходам чаще не выплачивают кредит (8,8 и 8,5% соответственно), чем клиенты с низким доходом (8,0%), одновеременно, клиенты с очень высоким доходом меньше всех подвержены вероятности невыплаты (7,1%).


- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#построим сводную таблицу
data_pivot = df.pivot_table(index = ['purpose'], columns = 'debt', values = 'gender', aggfunc = 'count').round(2)
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]) * 100, 1)
data_pivot.sort_values('ratio', ascending = False)

**Вывод**

Между целями кредита и вероятностью его задолженности есть взаимосвязь, но стоит отметить, что ее характер не явный и больше связан со скоростью возврата инвестированных средств и их размером.   
Кредит на недвижимость, как правило, выдается под более низкий процент, чем потребительские кредиты, кроме того, залогом по нему является сам объект собственности.
Поэтому клиент с большей вероятность выплатит кредит (вероятность невыплаты 7,2%).  
В тоже время кредит на свадьбу, как правило, на меньшую сумму и срок, что также снижает вероятность невыплаты (8,0%).
Вероятность невыплаты кредитов на образование и автомобиль отличается не значительно (9,2 и 9,4% соответственно).

## Шаг 4. Общий вывод

**Семейное положение и количество детей клиента на факт погашения кредита в срок.**

Исходная проблема: нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты анализа будут использованы для кредитного скоринга (оценки кредитоспособности клиента) банка.  
В результате исследования были подвтерждены следующие гипотезы:  
* `существует зависимость между наличием детей и вероятностью невыплаты кредита в срок`: при отсутствии детей ниже вероятность кредитной задолженности (7,5%), чем при их наличии (9,2%)
* `существует зависимость между семейным положением и вероятностью невыплаты кредита в срок`: наиболее дисциплинированными клиентами с низким риском кредитования являются  вдовцы/вдовицы (6,6%) и находящиеся в разводе (7,1%). Вероятность невыплаты ниже (и составляет 7,5%), если клиент находится в браке, чем если он не женат (не замужем) (9,8%) или состоит в незарегистрированном браке (9,3%).
* `существует зависимость между целью кредитования и вероятностью невыплаты кредита в срок`: наиболее низок риск выдачи кредита на недвижимость и свадьбу (7,2 и 8,0%), чем автокредитования и образование (9,4 и 9,2%).  
Уровень дохода на верноятность выплаты кредита не влияет.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.